<a href="https://colab.research.google.com/github/HAOalwayswin/------API/blob/main/Bllossom_AICA_gradio_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for Bllossom on Gradio!

## 01. Import all packages

In [1]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00


In [2]:
import gradio as gr
import os
import torch
from transformers import AutoProcessor, MllamaForConditionalGeneration
from PIL import Image

## 02. Load your model

In [3]:
# Determine the device (GPU if available, else CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# Load the model and processor
model_name = """Bllossom/llama-3.2-Korean-Bllossom-AICA-5B"""
model = MllamaForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map='cuda:0'
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/5.22k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/84.7k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.58G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/835M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [5]:
# Move the model to the appropriate device (GPU if available)
model.to(device)
processor = AutoProcessor.from_pretrained(model_name)
# VRAM을 많이 먹을 경우 아래 코드 실행
model.eval()

preprocessor_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/5.42k [00:00<?, ?B/s]

MllamaForConditionalGeneration(
  (vision_model): MllamaVisionModel(
    (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), padding=valid, bias=False)
    (gated_positional_embedding): MllamaPrecomputedPositionEmbedding(
      (tile_embedding): Embedding(9, 8197120)
    )
    (pre_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
      (embedding): Embedding(9, 5120)
    )
    (post_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
      (embedding): Embedding(9, 5120)
    )
    (layernorm_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (layernorm_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (transformer): MllamaVisionEncoder(
      (layers): ModuleList(
        (0-31): 32 x MllamaVisionEncoderLayer(
          (self_attn): MllamaVisionSdpaAttention(
            (q_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (k_proj): Linear(in_features=1280, out_features=1280, b

In [6]:
def predict(image=None, text=None):
  torch.cuda.empty_cache()
  with torch.no_grad():
      if image is not None:
        w,h = image.size
        if w < 500 or h < 500:
          image = image.resize((w*2,h*2))

      # LLM으로 사용할 때
      if image is None:
          messages = [
              {"role": "user", "content": [
                  {"type": "text", "text": text}  # Add the user-provided text input
              ]}
          ]
          inputs = processor.tokenizer.apply_chat_template(messages, add_generation_prompt=True,return_tensors='pt').to(model.device)
          outputs = model.generate(inputs, max_new_tokens=512,temperature=0.1,eos_token_id=processor.tokenizer.convert_tokens_to_ids('<|eot_id|>'))

      # VLM으로 사용할 때
      else:
          # Prepare the input messages
          messages = [
              {"role": "user", "content": [
                  {"type": "image"},  # Specify that an image is provided
                  {"type": "text", "text": text}  # Add the user-provided text input
              ]}
          ]
          # Create the input text using the processor's chat template
          input_text = processor.apply_chat_template(messages, add_generation_prompt=True)

          # Process the inputs and move to the appropriate device
          inputs = processor(image, input_text,add_special_tokens=False, return_tensors="pt").to(device)


      # Generate a response from the model
          with torch.cuda.amp.autocast():
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                use_cache=True,
                temperature=0.1,
                eos_token_id=processor.tokenizer.convert_tokens_to_ids('<|eot_id|>'),
                )

      # Decode the output to return the final response
      response = processor.decode(outputs[0])
      response = response[
          response.rindex('<|start_header_id|>assistant<|end_header_id|>\n\n')+\
          len('<|start_header_id|>assistant<|end_header_id|>\n\n'):].replace('<|eot_id|>','')

      return response

In [ ]:
# Define the Gradio interface
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Image(type="pil", label="Image Input"),  # Image input with label
        gr.Textbox(label="Text Input")  # Textbox input with label
    ],
    outputs=gr.Textbox(label="Generated Response"),  # Output with a more descriptive label
    title="Bllossom-AICA Demo",  # Title of the interface
    description="This demo uses Bllossom-AICA model to generate responses based on an image and text input.",  # Short description
    theme="compact",  # Using a compact theme for a cleaner look,
    batch=False,
)

# Launch the interface
interface.launch(debug=True, share=True)

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1115: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-67f7067a-44ea09fa0a19594e7ce5a1d0;4f27cbbc-b861-4c9e-8073-0946e1679ae7)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cf1684a6a28621845a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-6-06033fdd9259>:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-6-06033fdd9259>:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-6-06033fdd9259>:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-6-06033fdd9259>:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-6-06033fdd9259>:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-6-06033fdd9259>:36: Futur

# 제조업 수요 분석

## 1. 활용 실태
- 제조업은 공급자의 생산계획 및 소비자 구매변동과 관련이 있는 수요예측, 일정관리, 생산량 조절을 위해 기상정보를 활용.
  - 전반적 일정관리와 연관한 상시 주간예보, 단기예보 파악.
  - 엘니뇨 등의 기상현상에 따른 원자재 구매 관련 영향, 시장상황 등에 대한 자체 분석, 관리.
  - 일상적인 정보는 기상청정보를 이용하며, 기상이변에 따른 곡물가 변동 등의 이슈는 해외 기상정보로 파악.
- 계절상품에 대한 기획 등 활용의지는 있으나, 구체적인 기상정보 취득방법 및 기상정보에 대한 이해도는 낮음.

## 2. 수요 특성
- 일반적 관리를 위해서는 기상청 정보로 충분하다는 인식이 강함.
- 최근 6개월 전에 관련 계획을 수립·진행해야하므로 사전 구매계획(계약재배), 영업계획(예년 계절정보 반영) 등을 위한 전략적 기상정보 활용성 미흡.
- 계절별 상품개발, 수요예측 등을 위한 장기예보에 관심이 많으나, 기상예보의 정확도에 대한 신뢰가 낮아 활용성 미흡.
- 날씨리스크 자감을 위해 구매업체 분석, 일정조정 등으로 대응.

## 3. 수요충족 방안
- 활용가치를 인식하면 계속 구입할 의지는 있는 산업이나, 보다 구체적이고 입종 특화서비스 제공이 요구됨.
- 기상정보의 유효화는 원자재(식자재) 작황 예측, 수요예측, 생산관리에 직접적인 조언이 가능한 빅데이터 분석에 기반한 업체 특화 컨설팅을 통해 가능.
- 기상정보와 매출 간 상관분석에 대한 연구사업, 관련 설명회 등 홍보 필요.